# Code to Process Data from Marc

In [ ]:
lab_pool = set()
with open("data/Tungsten carbide data update.csv") as f:
    with open("co_train.csv", "w") as g:
        for line in f.readlines():
            if len(line) < 5:
                continue
            x = line[1:].split(",")[0]
            y = line.split("of")[1][:-4]
            g.write(f'"{x}",{y}\n')
            lab_pool.add(x)

In [ ]:
from itertools import product

wtr = range(4, 30)
ctempr = range(600, 1000 + 50, 50)
tempr = range(250, 350 + 25, 25)
dpwt = ["0.5 wt%", "5 wt%"]
dpotanti = [
    "with Co dopant metal",
    "with Fe dopant metal",
    "with Cu dopant metal",
    "with Pt dopant metal",
    "with Ni dopant metal",
]
dopantr = ["without any dopant metal"]
for w, d in product(dpwt, dpotanti):
    dopantr.append(f"{d} at {w}")

with open("co_pool.txt", "w") as f:
    for wt, ctemp, temp, dopant in product(wtr, ctempr, tempr, dopantr):
        prompt = (
            f"A {wt} wt% tungsten carbide catalyst was prepared {dopant} and carburized at {ctemp} °C. "
            f"The reaction was run at {temp} °C"
        )
        if prompt in lab_pool:
            print("skipping found one")
            continue
        f.write(prompt.split(",")[0] + "\n")

# Mult Prediction

In [ ]:
import bolift
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
from langchain.prompts.prompt import PromptTemplate


data_path = "data/co_train.csv"
np.random.seed(0)

In [ ]:
raw_data = pd.read_csv(data_path)
raw_data

In [ ]:
r = raw_data[raw_data.coyield == raw_data.coyield.max()]
print(r)
print(r.conditions.values)

In [ ]:
asktell = bolift.AskTellFewShotTopk(
    x_formatter=lambda x: f'the synthesis procedure:"{x}"',
    y_name="the CO yield",
    y_formatter=lambda y: f"{y:.2f}%",
    model="gpt-4",
    temperature=1,
    selector_k=5,
)

In [ ]:
for i, (c, y) in raw_data.iterrows():
    if i < len(raw_data) - 1:
        asktell.tell(c, y)

In [ ]:
asktell.predict(c), y

In [ ]:
# tell the last one
asktell.tell(c, y)

In [ ]:
s = asktell.inv_predict(20)
s

In [ ]:
with open("data/co_pool.txt") as f:
    pool = bolift.Pool(list(f.readlines()))
print(len(pool))

In [ ]:
result = asktell.ask(pool, "greedy", inv_filter=50)

In [ ]:
x = result[0][0]
print(x)

In [ ]:
dist = asktell.predict(x)
print(dist, dist.mean(), dist.std())

# GPR

In [ ]:
asktell = bolift.AskTellGPR(
    x_formatter=lambda x: f'synthesis procedure:"{x}"',
    y_name="CO yield",
    y_formatter=lambda y: f"{y:.2f}",
)

In [ ]:
for i, (c, y) in raw_data.iterrows():
    if i < len(raw_data) - 1:
        asktell.tell(c, y)

In [ ]:
asktell.predict(c), y

In [ ]:
# tell the last one
asktell.tell(c, y)

In [ ]:
with open("data/co_pool.txt") as f:
    pool = bolift.Pool(list(f.readlines()))

In [ ]:
result = asktell.ask(pool, "greedy")

In [ ]:
x = result[0][0]
print(x)

In [ ]:
dist = asktell.predict(x)
print(dist, dist.mean(), dist.std())